In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pip install pretty_midi

In [12]:
import os
import numpy as np
import pretty_midi
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from keras import layers

In [13]:
# Define some constants
SEQUENCE_LENGTH = 100 # The length of the input sequence for the RNN
BATCH_SIZE = 64 # The size of the mini-batch for training
EPOCHS = 10 # The number of epochs for training
EMBEDDING_DIM = 256 # The dimension of the embedding layer
RNN_UNITS = 512 # The number of units in the RNN layer
VOCAB_SIZE = 128 # The size of the vocabulary (number of MIDI notes)
NO_NOTE = 128  # Symbol for "no note"

# Define a function to load and preprocess the MIDI files
def load_midi_files(path):
  # Initialize an empty list to store the sequences
  sequences = []
  # Loop through all the files in the path
  for file in os.listdir(path):
    # Check if the file is a MIDI file
    if file.endswith(".mid"):
      # Load the MIDI file using pretty_midi
      midi = pretty_midi.PrettyMIDI(os.path.join(path, file))
      # Get the list of notes played in the MIDI file
      notes = [note.pitch for note in midi.instruments[0].notes]
      # Reshape the list of notes to create sub-sequences of length SEQUENCE_LENGTH
      num_sequences = len(notes) // SEQUENCE_LENGTH
      reshaped_notes = np.array(notes[:num_sequences * SEQUENCE_LENGTH])
      reshaped_notes = reshaped_notes.reshape(-1, SEQUENCE_LENGTH)
      # Append the sub-sequences to the list of sequences
      sequences.append(reshaped_notes)
  # Concatenate all the sub-sequences into one array
  sequences = np.concatenate(sequences, axis=0)
  # Pad the sequences with the symbol for "no note"
  sequences = pad_sequences(sequences, maxlen=SEQUENCE_LENGTH, value=NO_NOTE)
  # Return the array of sequences
  return sequences

# Path to your MIDI files
path_to_midi_files = "/content/drive/MyDrive/generate_audio/audio_files"

# Load the MIDI files
sequences = load_midi_files(path_to_midi_files)

# Split the sequences into training and validation sets
train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)

# Define a function to create and compile the RNN model
def create_model(vocab_size, embedding_dim, rnn_units):
  model = keras.Sequential()
  model.add(layers.Embedding(vocab_size + 1, embedding_dim))  # Add 1 to vocab_size because we added an extra symbol for "no note"
  model.add(layers.LSTM(rnn_units, return_sequences=True))
  model.add(layers.Dense(vocab_size + 1, activation="softmax"))  # Add 1 to vocab_size because we added an extra symbol for "no note"
  model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  return model

# Create and compile the RNN model using the defined constants
model = create_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS)

# Train the model on the training data for EPOCHS epochs and validate on the validation data
model.fit(train_sequences, train_sequences, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(val_sequences, val_sequences))


Epoch 1/10
4/4 [==============================] - 12s 3s/step - loss: 4.7985 - accuracy: 0.2622 - val_loss: 4.4109 - val_accuracy: 0.2277
Epoch 2/10
4/4 [==============================] - 8s 2s/step - loss: 4.3617 - accuracy: 0.1912 - val_loss: 4.0506 - val_accuracy: 0.1377
Epoch 3/10
4/4 [==============================] - 9s 2s/step - loss: 3.8243 - accuracy: 0.1890 - val_loss: 3.6926 - val_accuracy: 0.1836
Epoch 4/10
4/4 [==============================] - 8s 2s/step - loss: 3.5103 - accuracy: 0.2378 - val_loss: 3.4031 - val_accuracy: 0.2416
Epoch 5/10
4/4 [==============================] - 9s 2s/step - loss: 3.3466 - accuracy: 0.2753 - val_loss: 3.3617 - val_accuracy: 0.1913
Epoch 6/10
4/4 [==============================] - 10s 3s/step - loss: 3.2459 - accuracy: 0.2362 - val_loss: 3.2450 - val_accuracy: 0.3080
Epoch 7/10
4/4 [==============================] - 8s 2s/step - loss: 3.1515 - accuracy: 0.2899 - val_loss: 3.1867 - val_accuracy: 0.2638
Epoch 8/10
4/4 [=======================

In [ ]:
def generate_music(model, seed, num_notes):
  # Initialize an empty list to store the generated notes
  generated_notes = []
  # Loop for num_notes times
  for i in range(num_notes):
    # Predict the next note using the model and the seed sequence
    prediction = model.predict(seed)
    # Sample a note from the prediction using a multinomial distribution
    note = np.random.choice(range(VOCAB_SIZE + 1), p=prediction[0][-1])
    # Append the note to the list of generated notes
    generated_notes.append(note)
    # Update the seed sequence by appending the note and removing the first element
    seed = np.append(seed[:, 1:], [[note]], axis=1)
  # Return the array of generated notes
  return np.array(generated_notes)

# Generate new music using the trained model and a random seed sequence from the validation data
seed = val_sequences[np.random.randint(0, len(val_sequences))]
seed = seed.reshape(1, SEQUENCE_LENGTH)  # Reshape the seed sequence to have shape (1, SEQUENCE_LENGTH)
generated_notes = generate_music(model, seed, 1000)


In [15]:
!pip install MIDIUtil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for MIDIUtil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54570 sha256=cc0af5cb17d99ea38e32bda59f53a1d4aaf4179c534ca6c03bc3b479c77898c2
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built MIDIUtil


In [18]:
from midiutil import MIDIFile
from google.colab import files
from music21 import note, chord, instrument, stream

In [ ]:
def generate_music_midi(generated_notes):
    offset = 0
    output_notes = []
    for pattern in generated_notes:
        # assuming pattern is an integer
        new_note = note.Note(int(pattern))
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music.mid')

# Generate new music using the trained model and a random seed sequence from the validation data
seed = val_sequences[np.random.randint(0, len(val_sequences))]
seed = seed.reshape(1, SEQUENCE_LENGTH)  # Reshape the seed sequence to have shape (1, SEQUENCE_LENGTH)
generated_notes = generate_music(model, seed, 1000)

# Convert the generated notes to a MIDI file and save it
generate_music_midi(generated_notes)
